# Import Statements

In [ ]:
import numpy as np
import scipy
import random
from six.moves import range
# random.seed(5001)

from sklearn.metrics import f1_score, recall_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import pandas as pd
from scipy.signal import decimate
from scipy.stats import mode
import pywt
import seaborn as sn
from collections import Counter
from keras.utils import to_categorical

import warnings
warnings.filterwarnings('ignore')

# Data Extraction

In [ ]:
pa = pd.read_csv("Watch_accelerometer.csv")
# pa=pd.read_csv("Phones_gyroscope.csv")
# wa=pd.read_csv("Watch_accelerometer.csv")
# wg=pd.read_csv("Watch_gyroscope.csv")
print (pa.shape)
print("Done")

In [ ]:
pa = pa[pa['gt'] != 'null']
print (pa.shape)
pa = pa.dropna()
print (pa.shape)

In [ ]:
acts = pa['gt'].unique()
users = pa['User'].unique()
devices = pa['Device'].unique()
print (devices)
print (acts)
print (users)

In [ ]:
l = {}
for i, act in enumerate(acts):
    l[act] = i
print (l)

dev = {}
for i, d in enumerate(devices):
    dev[d] = i
print (dev)

use = {}
for i, u in enumerate(users):
    use[u] = i
print (use)

<b> 
Train users: a, b, c, d, e, f, g, h

Test user: i
</b>

In [ ]:
userTrain, userTest = [], []

for i in range(len(users)):
    train = []
    
    if i != 8:
        train.extend(users[i+1:])
    if i != 0:
        train.extend(users[:i])
    #print (users[i])
    userTrain.append(train)
    userTest.append(users[i])
    
print (userTrain, userTest)

In [ ]:
def getData(userList):

    devs, acc, labels = [], [], []

    for user in userList:
        pa_user = pa[pa['User'] == user]
        for act in acts:
            pa_act = pa_user[pa_user['gt'] == act]
            for device in devices:
                pa_dev = pa_act[pa_act['Device'] == device]
                pa_dev = pa_dev[['x', 'y','z']]
                if str(device) == 'lgwatch_1' or str(device) == 'lgwatch_2':
                    min_win = 400
                # elif str(device) == 'gear_1' or str(device) == 'gear_2':
                else:    
                    min_win = 200
                if(pa_dev.shape[0] >= min_win):
                    acc.append(pa_dev.values)
                    devs.append(device)
                    labels.append(l[act])
        print (len(labels))
        print (f'{user} done')
    
    acc = np.array(acc)
    labels = np.array(labels)
    devs = np.array(devs)
    print ("Done")
    print(acc.shape, labels.shape, devs.shape)
    return acc, labels, devs

Xtrain1, Ytrain1, devs1 = getData(userTrain[-1])
Xtest1, Ytest1, devs2 = getData(userTest[-1])

# Getting the Windowed Data

In [ ]:
def getWindowedData(acc, labels, index, w_min):
    
    windowData, windowLabels = [], []
    num_windows = acc[index].shape[0] // w_min
    if num_windows == 0:
        print(acc[index].shape[0], w_min)
    k = 0
    for _ in range(num_windows):
        windowData.append(acc[index][k:k+w_min])
        k += w_min
        windowLabels.append(labels[index])
    return windowData, windowLabels

In [ ]:
# Getting 2 seconds (200 samples) of data for all devices

main_train = []
for i in range(len(Xtrain1)):
    if str(devs1[i]) == 'lgwatch_1' or str(devs1[i]) == 'lgwatch_2':
        w_min = 400
    else:
        w_min = 200
    main_train.append((getWindowedData(Xtrain1, Ytrain1, i, w_min)))

main_test = []
for i in range(len(Xtest1)):
    if str(devs2[i]) == 'lgwatch_1' or str(devs2[i]) == 'lgwatch_2':
        w_min = 400
    else:
        w_min = 200
    main_test.append((getWindowedData(Xtest1, Ytest1, i, w_min)))

In [ ]:
print (main_train[-1][0][0].shape)
print (main_train[-1][1][0])

# Decimating the Windowed Data

In [ ]:
def decimateThatSignal(main, i):
        final_decimated_signals = []
        label = []
        if main[i][0][0].shape[0] == 400:
            for j in range(len(main[i][0])):
                decimated_signal_0 = decimate(main[i][0][j][:,0], 2)
                decimated_signal_1 = decimate(main[i][0][j][:,1], 2)
                decimated_signal_2 = decimate(main[i][0][j][:,2], 2)
                decimated_signal = np.dstack((decimated_signal_0, decimated_signal_1, decimated_signal_2))
                final_decimated_signals.append(decimated_signal)
                label.append(main[i][1][j])
            return np.array(final_decimated_signals), np.array(label)
        else:
            return np.array(main[i][0]), np.array(main[i][1])

In [ ]:
w_min = 200

Xtrain1 = decimateThatSignal(main_train, 0)[0].reshape((-1, w_min, 3))
Ytrain1 = decimateThatSignal(main_train, 0)[1]
for i in range(1, len(main_train)):
    print (i)
    Xtrain1, Ytrain1 = np.vstack((Xtrain1, decimateThatSignal(main_train, i)[0].reshape((-1, w_min, 3)))), np.hstack((Ytrain1, decimateThatSignal(main_train, i)[1]))
    #mainDecimated.append(DecimateThatSignal(i).reshape((-1, 100, 3)))
Xtrain1 = np.array(Xtrain1)
Ytrain1 = np.array(Ytrain1)

In [ ]:
w_min = 200

Xtest1 = decimateThatSignal(main_test, 0)[0].reshape((-1, w_min, 3))
Ytest1 = decimateThatSignal(main_test, 0)[1]
for i in range(1, len(main_test)):
    print (i)
    Xtest1, Ytest1 = np.vstack((Xtest1, decimateThatSignal(main_test, i)[0].reshape((-1, w_min, 3)))), np.hstack((Ytest1, decimateThatSignal(main_test, i)[1]))
    #mainDecimated.append(DecimateThatSignal(i).reshape((-1, 100, 3)))
Xtest1 = np.array(Xtest1)
Ytest1 = np.array(Ytest1)

In [ ]:
print (Xtrain1.shape, Ytrain1.shape)
print (Xtest1.shape, Ytest1.shape)

# Getting the DWTed Data

In [ ]:
pywt.wavelist()

In [ ]:
# Change window size here as and when DWT wavelet changes

w_min = 100+3

In [ ]:
def performDWT(x, Y):
    masterX = []
    for i in range(len(x)):
        Xca, Xda = pywt.dwt(x[i].reshape((-1, 3))[:,0], 'db4', mode='periodic')
        Yca, Yda = pywt.dwt(x[i].reshape((-1, 3))[:,1], 'db4', mode='periodic')
        Zca, Zda = pywt.dwt(x[i].reshape((-1, 3))[:,2], 'db4', mode='periodic')
        coef = np.hstack((Xca, Yca, Zca)).reshape((-1, w_min, 3))
        masterX.append((coef, Y[i]))
        print (i)
    masterX = np.array(masterX)
    return masterX

In [ ]:
masterTrain = performDWT(Xtrain1, Ytrain1)
masterTest = performDWT(Xtest1, Ytest1)

In [ ]:
print (masterTrain.shape, masterTest.shape)

# Train-Test Split

In [ ]:
#for training
Y_train = []
X_train = np.zeros((masterTrain.shape[0], 1, w_min, 3))
for i in range(masterTrain.shape[0]):
    X_train[i, :, :] = masterTrain[i][0][:]
    Y_train.append(masterTrain[i][1])
y_train = np.array(Y_train)   


#for test
Y_test = []
X_test = np.zeros((masterTest.shape[0], 1, w_min, 3))
for i in range(masterTest.shape[0]):
    X_test[i, :, :] = masterTest[i][0][:]
    Y_test.append(masterTest[i][1])
y_test = np.array(Y_test)

In [ ]:
print (y_train.shape, y_test.shape)

In [ ]:
X_train = X_train.reshape((-1, w_min, 3)).astype('float32')
X_test = X_test.reshape((-1, w_min, 3)).astype('float32')

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Params Initialize

In [ ]:
num_classes = 6
img_rows, img_cols = 103, 3
learning_rate = 2e-4
# subset_size = 2000

# Train-Valid-Pool Split

In [ ]:
np.bincount(y_test)

In [ ]:
X_test, X_pool, y_test, y_pool = train_test_split(X_test, y_test, test_size=0.7, stratify=y_test)

print ('Pool:', X_pool.shape, y_pool.shape)
print ('Test:', X_test.shape, y_test.shape)

# Zero-Mean Normalization

In [ ]:
# X_train_All, X_test, y_train_All, y_test = train_test_split(data, labels, stratify=labels, test_size=0.2, random_state=5233)

#X_train, X_test, y_train_All, y_test = train_test_split(data, labels, stratify=labels, test_size=0.2, random_state=5233)

X_train_fit0 = StandardScaler().fit(X_train[:, :, 0])
X_train0 = X_train_fit0.transform(X_train[:, :, 0])

X_train_fit1 = StandardScaler().fit(X_train[:, :, 1])
X_train1 = X_train_fit1.transform(X_train[:, :, 1])

X_train_fit2 = StandardScaler().fit(X_train[:, :, 2])
X_train2 = X_train_fit2.transform(X_train[:, :, 2])

X_pool0 = X_train_fit0.transform(X_pool[:, :, 0])

X_pool1 = X_train_fit1.transform(X_pool[:, :, 1])

X_pool2 = X_train_fit2.transform(X_pool[:, :, 2])

X_test0 = X_train_fit0.transform(X_test[:, :, 0])

X_test1 = X_train_fit1.transform(X_test[:, :, 1])

X_test2 = X_train_fit2.transform(X_test[:, :, 2])

# print (X_train0.shape, X_test0.shape)

X_train = np.dstack((X_train0, X_train1, X_train2))
X_pool = np.dstack((X_pool0, X_pool1, X_pool2))
X_test = np.dstack((X_test0, X_test1, X_test2))

del X_train0, X_train1, X_train2, X_pool0, X_pool1, X_pool2, X_test0, X_test1, X_test2

print (X_train.shape, X_pool.shape, X_test.shape)
print (y_train.shape, y_pool.shape, y_test.shape)

In [ ]:
print ('Distribution of Training Classes:', np.bincount(y_train))

Y_train = to_categorical(y_train, num_classes)
Y_test = to_categorical(y_test, num_classes)
Y_pool = to_categorical(y_pool, num_classes)

In [ ]:
print ('Distribution of Train Classes:', np.bincount(y_train))
print ('Distribution of Pool Classes:', np.bincount(y_pool))
print ('Distribution of Test Classes:', np.bincount(y_test))

In [ ]:
print ('Train:', X_train.shape, Y_train.shape)
print ('Pool:', X_pool.shape, Y_pool.shape)
print ('Test:', X_test.shape, Y_test.shape)

# Keras Imports

In [ ]:
import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Input, Flatten, Reshape, concatenate, Lambda
from keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, BatchNormalization
from keras.utils import to_categorical
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

from acquisition_functions import bald, varratio, maxentropy, random_acq, segnet

In [ ]:
X_train0 = X_train[:, :, 0].reshape((-1, w_min, 1))
X_train1 = X_train[:, :, 1].reshape((-1, w_min, 1))
X_train2 = X_train[:, :, 2].reshape((-1, w_min, 1))

X_pool0 = X_pool[:, :, 0].reshape((-1, w_min, 1))
X_pool1 = X_pool[:, :, 1].reshape((-1, w_min, 1))
X_pool2 = X_pool[:, :, 2].reshape((-1, w_min, 1))

X_test0 = X_test[:, :, 0].reshape((-1, w_min, 1))
X_test1 = X_test[:, :, 1].reshape((-1, w_min, 1))
X_test2 = X_test[:, :, 2].reshape((-1, w_min, 1))

print (X_train0.shape, X_pool0.shape, X_test0.shape)

In [ ]:
def harnet():

    # Model X
    inputX = Input(shape=(X_train0.shape[1], X_train0.shape[2]))

    convX1 = Conv1D(filters=8, kernel_size=2, padding='same', activation='relu')(inputX)
    batchX1 = BatchNormalization()(convX1)
    poolX1 = MaxPooling1D(pool_size=2, padding='same')(batchX1)

    convX2 = Conv1D(filters=16, kernel_size=2, padding='same', activation='relu')(poolX1)
    batchX2 = BatchNormalization()(convX2)
    poolX2 = MaxPooling1D(pool_size=2, padding='same')(batchX2)

    modelX = Flatten()(poolX2)

    # Model Y
    inputY = Input(shape=(X_train1.shape[1], X_train2.shape[2]))

    convY1 = Conv1D(filters=8, kernel_size=2, padding='same', activation='relu')(inputY)
    batchY1 = BatchNormalization()(convY1)
    poolY1 = MaxPooling1D(pool_size=2, padding='same')(batchY1)

    convY2 = Conv1D(filters=16, kernel_size=2, padding='same', activation='relu')(poolY1)
    batchY2 = BatchNormalization()(convY2)
    poolY2 = MaxPooling1D(pool_size=2, padding='same')(batchY2)

    modelY = Flatten()(poolY2)

    # Model Z
    inputZ = Input(shape=(X_train2.shape[1], X_train2.shape[2]))

    convZ1 = Conv1D(filters=8, kernel_size=2, padding='same', activation='relu')(inputZ)
    batchZ1 = BatchNormalization()(convZ1)
    poolZ1 = MaxPooling1D(pool_size=2, padding='same')(batchZ1)

    convZ2 = Conv1D(filters=16, kernel_size=2, padding='same', activation='relu')(poolZ1)
    batchZ2 = BatchNormalization()(convZ2)
    poolZ2 = MaxPooling1D(pool_size=2, padding='same')(batchZ2)

    # Merge Models X, Y, Z
    modelZ = Flatten()(poolZ2)

    merged_model = concatenate([modelX, modelY, modelZ])
    print (K.int_shape(merged_model))

    final_merge = Reshape((K.int_shape(merged_model)[1]//3, 3, 1))(merged_model)
    print (K.int_shape(final_merge))

    # Conv2D
    conv1 = Conv2D(filters=8, kernel_size=(3, 3), padding='same')(final_merge)
    batch1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), padding='same')(batch1)

    conv2 = Conv2D(filters=16, kernel_size=(3, 3), padding='same')(pool1)
    batch2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2), padding='same')(batch2)

    #drop2d = Lambda(lambda x: K.dropout(x, level=0.25))(pool2)

    flatten = Flatten()(pool2)

    # Dense
    fc1 = Dense(16, activation='relu', kernel_initializer='glorot_normal')(flatten)
    # fc1 = Dropout(0.25)(fc1)
    # Stochastic Dropout Layer
    fc1 = Lambda(lambda x: K.dropout(x, level=0.25))(fc1)
    fc2 = Dense(8, activation='relu')(flatten)

    # Output Layer - Softmax
    output = Dense(num_classes, activation='softmax')(fc2)
    
    # Final Model
    model = Model([inputX, inputY, inputZ], output)
    
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate), metrics=['accuracy']) #beta_1=0.9, beta_2=0.999))
    
    return model

In [ ]:
model = harnet()
model.summary()

In [ ]:
# save_path = './Save_Files/Watch_User_i.h5'
# checkpoint = ModelCheckpoint(save_path, monitor='val_acc', save_best_only=True, mode='max')
# callback = [checkpoint]

# hist = model.fit([X_train0, X_train1, X_train2], Y_train, epochs=50, batch_size=64, \
#                  validation_data=([X_test0, X_test1, X_test2], Y_test))#, callbacks=callback)

In [ ]:
model = load_model('./Save_Files/Watch_User_i.h5')

In [ ]:
# np.save('./Save_Files/hist_Watch_User_i', hist.history)
histC = np.load('./Save_Files/hist_Watch_User_i.npy').tolist()
print (len(histC['val_loss']))

In [ ]:
# print (max(hist.history['val_acc']))
print (max(histC['val_acc']))
model.evaluate([X_test0, X_test1, X_test2], Y_test)

In [ ]:
y_prob = model.predict([X_test0, X_test1, X_test2])
y_pred = y_prob.argmax(axis=-1)

y_true = np.array([np.argmax(y) for y in Y_test])

In [ ]:
print (y_true.shape, y_pred.shape)

# Preprocess for Active Learning

In [ ]:
subset_size = X_pool.shape[0]

subset_indices = np.asarray(random.sample(range(0, X_pool.shape[0]), subset_size))
X_pool_subset = X_pool[subset_indices]
Y_pool_subset = Y_pool[subset_indices]
# X_pool_new = np.delete(X_pool, subset_indices, axis=0)
# Y_pool_new = np.delete(Y_pool, subset_indices, axis=0)

print (X_pool_subset.shape, Y_pool_subset.shape)
# print (X_pool_new.shape, Y_pool_new.shape)

In [ ]:
X_pool_subset0 = X_pool_subset[:, :, 0].reshape((-1, w_min, 1))
X_pool_subset1 = X_pool_subset[:, :, 1].reshape((-1, w_min, 1))
X_pool_subset2 = X_pool_subset[:, :, 2].reshape((-1, w_min, 1))

print (X_pool_subset0.shape, X_pool_subset1.shape, X_pool_subset2.shape)

In [ ]:
pool_min = 0
pool_max = int(0.6*X_pool_subset.shape[0])
pool_iter = pool_max//8

print (pool_min, pool_max, pool_iter)

In [ ]:
pool_percents = np.arange(12.5, 101, 12.5)
print (pool_percents)

# BALD

In [ ]:
acc_inc_bald, f1_inc_bald, recall_inc_bald = [], [], []

# Acquisition queries 8 times
for n_queries in range(pool_min, pool_max, pool_iter)[1:]:
    
    print ('Queries:', n_queries)
    model = load_model('./Save_Files/Watch_User_i.h5')

    bald_uncertainty_estimates = bald([X_pool_subset0, X_pool_subset1, X_pool_subset2], \
                                      num_classes, model, 16, 10)

    bald_uncertainty_estimates = bald_uncertainty_estimates.flatten()
    acquired_indices = bald_uncertainty_estimates.argsort()[-n_queries:][::-1]

    X_acquired_bald = X_pool[acquired_indices]
    Y_acquired_bald = Y_pool[acquired_indices]

    print ('BALD Acquired Shape:', X_acquired_bald.shape, Y_acquired_bald.shape)

    y_acquired_bald = Y_acquired_bald.argmax(axis=-1)
    np.bincount(y_acquired_bald)

    # Incremental Learning
    X_acquired0 = X_acquired_bald[:, :, 0].reshape((-1, w_min, 1))
    X_acquired1 = X_acquired_bald[:, :, 1].reshape((-1, w_min, 1))
    X_acquired2 = X_acquired_bald[:, :, 2].reshape((-1, w_min, 1))

    print (X_acquired0.shape, X_acquired1.shape, X_acquired2.shape)

    hist = model.fit([X_acquired0, X_acquired1, X_acquired2], Y_acquired_bald, epochs=30, batch_size=4, \
              validation_data=([X_test0, X_test1, X_test2], Y_test))
    inc_acc = max(hist.history['val_acc'])
    model.evaluate([X_test0, X_test1, X_test2], Y_test, batch_size=8)
    print ('Incremental Learning:', inc_acc)
    
    acc_inc_bald.append(inc_acc)
    y_prob = model.predict([X_test0, X_test1, X_test2], batch_size=8)
    y_pred = y_prob.argmax(axis=-1)
    f1_inc_bald.append(f1_score(y_test, y_pred, average='weighted'))
    recall_inc_bald.append(recall_score(y_test, y_pred, average='weighted'))
    
    print ()

# Var Ratio

In [ ]:
acc_inc_var, f1_inc_var, recall_inc_var = [], [], []

# Acquisition queries 8 times
for n_queries in range(pool_min, pool_max, pool_iter)[1:]:
    
    print ('Queries:', n_queries)
    model = load_model('./Save_Files/Watch_User_i.h5')

    var_uncertainty_estimates = varratio([X_pool_subset0, X_pool_subset1, X_pool_subset2], \
                                      num_classes, model, 16, 10)

    var_uncertainty_estimates = var_uncertainty_estimates.flatten()
    acquired_indices = var_uncertainty_estimates.argsort()[-n_queries:][::-1]

    X_acquired_var = X_pool[acquired_indices]
    Y_acquired_var = Y_pool[acquired_indices]

    print ('Var Ratio Acquired Shape:', X_acquired_var.shape, Y_acquired_var.shape)

    y_acquired_var = Y_acquired_var.argmax(axis=-1)
    np.bincount(y_acquired_var)

    # Incremental Learning
    X_acquired0 = X_acquired_var[:, :, 0].reshape((-1, w_min, 1))
    X_acquired1 = X_acquired_var[:, :, 1].reshape((-1, w_min, 1))
    X_acquired2 = X_acquired_var[:, :, 2].reshape((-1, w_min, 1))

    print (X_acquired0.shape, X_acquired1.shape, X_acquired2.shape)

    hist = model.fit([X_acquired0, X_acquired1, X_acquired2], Y_acquired_var, epochs=30, batch_size=4, \
              validation_data=([X_test0, X_test1, X_test2], Y_test))
    inc_acc = max(hist.history['val_acc'])
    model.evaluate([X_test0, X_test1, X_test2], Y_test, batch_size=8)
    print ('Incremental Learning:', inc_acc)
    
    acc_inc_var.append(inc_acc)
    y_prob = model.predict([X_test0, X_test1, X_test2], batch_size=8)
    y_pred = y_prob.argmax(axis=-1)
    f1_inc_var.append(f1_score(y_test, y_pred, average='weighted'))
    recall_inc_var.append(recall_score(y_test, y_pred, average='weighted'))
    
    print ()

# Max Entropy

In [ ]:
acc_inc_maxent, f1_inc_maxent, recall_inc_maxent = [], [], []

# Acquisition queries 8 times
for n_queries in range(pool_min, pool_max, pool_iter)[1:]:
    
    print ('Queries:', n_queries)
    model = load_model('./Save_Files/Watch_User_i.h5')

    maxent_uncertainty_estimates = maxentropy([X_pool_subset0, X_pool_subset1, X_pool_subset2], \
                                      num_classes, model, 16, 10)

    maxent_uncertainty_estimates = maxent_uncertainty_estimates.flatten()
    acquired_indices = maxent_uncertainty_estimates.argsort()[-n_queries:][::-1]

    X_acquired_maxent = X_pool[acquired_indices]
    Y_acquired_maxent = Y_pool[acquired_indices]

    print ('Max Entropy Acquired Shape:', X_acquired_maxent.shape, Y_acquired_maxent.shape)

    y_acquired_maxent = Y_acquired_maxent.argmax(axis=-1)
    np.bincount(y_acquired_maxent)

    # Incremental Learning
    X_acquired0 = X_acquired_maxent[:, :, 0].reshape((-1, w_min, 1))
    X_acquired1 = X_acquired_maxent[:, :, 1].reshape((-1, w_min, 1))
    X_acquired2 = X_acquired_maxent[:, :, 2].reshape((-1, w_min, 1))

    print (X_acquired0.shape, X_acquired1.shape, X_acquired2.shape)

    hist = model.fit([X_acquired0, X_acquired1, X_acquired2], Y_acquired_maxent, epochs=30, batch_size=4, \
              validation_data=([X_test0, X_test1, X_test2], Y_test))
    inc_acc = max(hist.history['val_acc'])
    model.evaluate([X_test0, X_test1, X_test2], Y_test, batch_size=8)
    print ('Incremental Learning:', inc_acc)
    
    acc_inc_maxent.append(inc_acc)
    y_prob = model.predict([X_test0, X_test1, X_test2], batch_size=8)
    y_pred = y_prob.argmax(axis=-1)
    f1_inc_maxent.append(f1_score(y_test, y_pred, average='weighted'))
    recall_inc_maxent.append(recall_score(y_test, y_pred, average='weighted'))
    
    print ()

# Random Acquisitions

In [ ]:
acc_inc_rand, f1_inc_rand, recall_inc_rand = [], [], []

# Acquisition queries 8 times
for n_queries in range(pool_min, pool_max, pool_iter)[1:]:
    
    print ('Queries:', n_queries)
    model = load_model('./Save_Files/Watch_User_i.h5')

    rand_uncertainty_estimates = random_acq([X_pool_subset0, X_pool_subset1, X_pool_subset2], \
                                      num_classes, model, 16, 10)

    rand_uncertainty_estimates = rand_uncertainty_estimates.flatten()
    acquired_indices = rand_uncertainty_estimates.argsort()[-n_queries:][::-1]

    X_acquired_rand = X_pool[acquired_indices]
    Y_acquired_rand = Y_pool[acquired_indices]

    print ('Random Acquired Shape:', X_acquired_rand.shape, Y_acquired_rand.shape)

    y_acquired_rand = Y_acquired_rand.argmax(axis=-1)
    np.bincount(y_acquired_rand)

    # Incremental Learning
    X_acquired0 = X_acquired_rand[:, :, 0].reshape((-1, w_min, 1))
    X_acquired1 = X_acquired_rand[:, :, 1].reshape((-1, w_min, 1))
    X_acquired2 = X_acquired_rand[:, :, 2].reshape((-1, w_min, 1))

    print (X_acquired0.shape, X_acquired1.shape, X_acquired2.shape)

    hist = model.fit([X_acquired0, X_acquired1, X_acquired2], Y_acquired_rand, epochs=60, batch_size=16, \
              validation_data=([X_test0, X_test1, X_test2], Y_test))
    inc_acc = max(hist.history['val_acc'])
    model.evaluate([X_test0, X_test1, X_test2], Y_test, batch_size=8)
    print ('Incremental Learning:', inc_acc)
    
    acc_inc_rand.append(inc_acc)
    y_prob = model.predict([X_test0, X_test1, X_test2], batch_size=8)
    y_pred = y_prob.argmax(axis=-1)
    f1_inc_rand.append(f1_score(y_test, y_pred, average='weighted'))
    recall_inc_rand.append(recall_score(y_test, y_pred, average='weighted'))
    
    print ()

In [ ]:
acc_inc_df = pd.DataFrame(index=np.arange(pool_min, pool_max, pool_iter)[1:])

acc_inc_df['BALD'] = np.array(acc_inc_bald) * 100
acc_inc_df['VarRatio'] = np.array(acc_inc_var) * 100
acc_inc_df['MaxEntropy'] = np.array(acc_inc_maxent) * 100
acc_inc_df['Random'] = np.array(acc_inc_rand) * 100

acc_inc_df

In [ ]:
f1_inc_df = pd.DataFrame(index=np.arange(pool_min, pool_max, pool_iter)[1:])

f1_inc_df['BALD'] = np.array(f1_inc_bald) * 100
f1_inc_df['VarRatio_'] = np.array(f1_inc_var) * 100
f1_inc_df['MaxEntropy'] = np.array(f1_inc_maxent) * 100
f1_inc_df['Random'] = np.array(f1_inc_rand) * 100

f1_inc_df

In [ ]:
recall_inc_df = pd.DataFrame(index=np.arange(pool_min, pool_max, pool_iter)[1:])

recall_inc_df['BALD'] = np.array(recall_inc_bald) * 100
recall_inc_df['VarRatio_'] = np.array(recall_inc_var) * 100
recall_inc_df['MaxEntropy'] = np.array(recall_inc_maxent) * 100
recall_inc_df['Random'] = np.array(recall_inc_rand) * 100

recall_inc_df

In [ ]:
acc_inc_df.to_csv('./CSV_Results/User_i/acc_inc.csv', sep=',')
f1_inc_df.to_csv('./CSV_Results/User_i/f1_inc.csv', sep=',')
recall_inc_df.to_csv('./CSV_Results/User_i/recall_inc.csv', sep=',')